# Your First TrackStar Program

In this tutorial, we'll demonstrate a simple use case of TrackStar involving synthetic data to illustrate a typical program.
We'll start by simply importing TrackStar and NumPy.

In [1]:
import numpy as np
import trackstar

/Users/jamesjohnson/Work/Research/lib/public-dev/TrackStar/trackstar/version.py:91: VersionWarning: Using a development version of TrackStar. Be advised that the features in development may exhibit buggy behavior and/or numerical artifacts. We encourage consulting with TrackStar developers.
  if self.dev is not None: warnings.warn("""\
/Users/jamesjohnson/Work/Research/lib/public-dev/TrackStar/trackstar/version.py:95: VersionWarning: Using an un-released version of TrackStar.
  if not self.isreleased: warnings.warn("""\


## A Mock Data Sample

Let's set up our mock sample.
For illustrative purposes, we'll keep it simple and straight-forward.

We'll take $N = 100$ points along the line $x = y = z$ in 3-dimensional space, following an intrinsic distribution that follows a Gaussian centered on zero (i.e. $\langle x \rangle = \langle y \rangle = \langle z \rangle = 0$) with a standard deviation of $\sigma = 1$.
To demonstrate TrackStar's user-friendliness in fitting non-uniform samples, we'll let only *half* of our data vectors have measurements of $z$.
Such instances may arise, e.g., within astrophysics, when not every star has a reliable age measurement.

We'll let $x$ and $y$ have measurement uncertainties of $\sigma_x = \sigma_y = 0.1$, while each measurement of $z$ will have a more substantial uncertainty of $\sigma_z = 0.3$.
In general, $z$ is a stand-in for some quantity that may be challenging to measure, where real samples may have only a handful of coarse measurements, but the information is nonetheless valuable.

Before bringing in TrackStar, let's set up the numbers themselves:

In [2]:
# the underlying sample with no measurement uncertainty
true_values = np.random.normal(loc = 0, scale = 1, size = 100)

# the sampled true values perturbed my measurement uncertainty for x and y
x = true_values + np.random.normal(loc = 0, scale = 0.1, size = 100)
y = true_values + np.random.normal(loc = 0, scale = 0.1, size = 100)

# half of the z measurements are missing
z = np.array([true_values[i] + np.random.normal(
    loc = 0, scale = 0.3) if i % 2 else float("nan") for i in range(100)])

To construct these arrays, we made use of NumPy's automatic component-wise addition when the arrays are the same length.
Let's inspect the first few elements of each of them:

In [3]:
print("x: ", x[:5])
print("y: ", y[:5])
print("z: ", z[:5])

x:  [ 0.4216244   0.68791544  1.67191643  1.05889879 -0.12395773]
y:  [0.53783069 0.93261433 1.97318748 0.94047328 0.1018157 ]
z:  [       nan 0.89020912        nan 0.85676598        nan]


TrackStar recognizes ``NaN`` values as missing data, so every other value of $z$ is assigned ``NaN`` accordingly (more on this below).

### ``trackstar.sample``: The Workhorse for Storing Data

The workhorse for storing data in TrackStar is ``sample``, which stores data vectors in a dictionary-like manner, using string labels to denote different measured quantities.
The most straight-forward way to construct one is to give it a dictionary containing the arrays of each measured quantity:

In [4]:
data = trackstar.sample({
    "x": x,
    "y": y,
    "z": z,
    "err_x": 100 * [0.1],
    "err_y": 100 * [0.1],
    "err_z": [0.3 if i % 2 else float("nan") for i in range(100)]
})
print(data)

sample(
    N = 100
    x --------------> [ 4.2162e-01,  6.8792e-01,  1.6719e+00, ...,  7.2159e-01,  1.0742e+00, -5.5068e-01]
    y --------------> [ 5.3783e-01,  9.3261e-01,  1.9732e+00, ...,  6.7938e-01,  1.0338e+00, -6.4173e-01]
    z --------------> [ nan       ,  8.9021e-01,  nan       , ...,  1.0793e+00,  nan       ,  4.9216e-01]
)


Let's break down the above call to ``trackstar.sample``.
We gave it one parameter, a dictionary, containing each of our arrays ``x``, ``y``, and ``z`` containing our data, each with an intuitive label.
Within this dictionary, we also gave it the associated measurement uncertainties ``err_x``, ``err_y``, and ``err_z``.
There are additional ways to load your data into a ``sample``, which we'll cover in another tutorial.
It is also possible to give your sample additional information on each data vector, separate from the measurements themselves (see [extra information for your ``sample``](#extra-information-for-your-sample) below).

What was returned is a data structure with similar behavior to the ``pandas DataFrame`` in that it can be indexed with both row number and column label.
By calling the ``.keys()`` instance method, we are able to access the labels that we have given our data.

In [5]:
print(data.keys())

['x', 'y', 'z']


TrackStar has recognized the measurement uncertainties for what they are based on their labels in our call to ``trackstar.sample`` above (see [discussion below](#where-the-measurement-uncertainties-reside) for information on where they've gone).

We can also access the number of data vectors in our sample by calling either ``len(data)`` or ``data.size``:

In [6]:
print(len(data))
print(data.size)

100
100


By indexing with a row number, we get each component of the same data vector:

In [7]:
print(data[0])
print(data[1])

datum(
    x --------------> 4.2162e-01
    y --------------> 5.3783e-01
)
datum(
    x --------------> 6.8792e-01
    y --------------> 9.3261e-01
    z --------------> 8.9021e-01
)


Note that the ``NaN`` value we entered for our zero'th data vector does not show up here, because TrackStar has recognized this as an indication that there is no measurement.
There are some caveats associated with this behavior, which we recommend new users familiarize themselves with (see the [note on NaNs in TrackStar](#an-important-note-on-nans-in-trackstar) below).
In short, TrackStar does not allow ``NaN`` values to be changed to numerical values, and vice versa, because they do not correspond to actual blocks of memory storing data vector components.

By indexing with a column label, we get the measurements of that quantity for each data vector:

In [8]:
print(data["x"])
print(data["z"])

linked_array([ 4.2162e-01,  6.8792e-01,  1.6719e+00, ...,  7.2159e-01,  1.0742e+00, -5.5068e-01])
linked_array([ nan       ,  8.9021e-01,  nan       , ...,  1.0793e+00,  nan       ,  4.9216e-01])


We've received a particular type of ``array`` as output, namely a ``linked_array``.
This class of array-like objects is special in that its memory is "linked" with our ``sample`` in the sense that modifying its elements *also* modifies the sample.
For example:

In [9]:
x = data["x"]
old_value = x[0]
x[0] = 0
print(x)
print(data)
x[0] = old_value
print(data)

linked_array([ 0.0000e+00,  6.8792e-01,  1.6719e+00, ...,  7.2159e-01,  1.0742e+00, -5.5068e-01])
sample(
    N = 100
    x --------------> [ 0.0000e+00,  6.8792e-01,  1.6719e+00, ...,  7.2159e-01,  1.0742e+00, -5.5068e-01]
    y --------------> [ 5.3783e-01,  9.3261e-01,  1.9732e+00, ...,  6.7938e-01,  1.0338e+00, -6.4173e-01]
    z --------------> [ nan       ,  8.9021e-01,  nan       , ...,  1.0793e+00,  nan       ,  4.9216e-01]
)
sample(
    N = 100
    x --------------> [ 4.2162e-01,  6.8792e-01,  1.6719e+00, ...,  7.2159e-01,  1.0742e+00, -5.5068e-01]
    y --------------> [ 5.3783e-01,  9.3261e-01,  1.9732e+00, ...,  6.7938e-01,  1.0338e+00, -6.4173e-01]
    z --------------> [ nan       ,  8.9021e-01,  nan       , ...,  1.0793e+00,  nan       ,  4.9216e-01]
)


By modifying the ``linked_list`` that we received by indexing ``data["x"]``, we are also modifying ``data`` itself.
Changes to one of these variables also affects the other.

We can also index the sample with both row number and column label simultaneously:

In [10]:
print(data["x", 0])
print(data[1, "y"])
print(data["z", 0])

0.4216244016571581
0.932614333382359
nan


In general, we recommend indexing with the rule ``data[row, column]`` as opposed to ``data[row][column]`` as it is both faster and more memory efficient.
For non-uniform samples, the ``data[row][column]`` rule may also result in a seemingly erroneous ``KeyError`` if a particular data vector does not have a measurement for the quantity in question (see [further discussion](#beware-when-indexing-non-uniform-samples) below), whereas ``data[row, column]`` would return a ``NaN`` value.

### Where the Measurement Uncertainties Reside

When we called ``trackstar.sample`` above, the dictionary keys labeled ``"err_x"``, ``"err_y"``, and ``"err_z"`` were excluded from the sample and not treated as data vector components.
TrackStar will treat any keys beginning with ``"err_"`` or ending in ``"_err"`` as measurement uncertainties and automatically construct diagonalized covariance matrices for each data vector:

In [11]:
print(data[0].cov)
print(data[1].cov)

covariance matrix(
    [ 1.0000e-02,  0.0000e+00]
    [ 0.0000e+00,  1.0000e-02]
    Quantities: [x, y] (in the order of indexing)
)
covariance matrix(
    [ 1.0000e-02,  0.0000e+00,  0.0000e+00]
    [ 0.0000e+00,  1.0000e-02,  0.0000e+00]
    [ 0.0000e+00,  0.0000e+00,  9.0000e-02]
    Quantities: [x, y, z] (in the order of indexing)
)


If we had not specified any measurement uncertainties in our call to ``trackstar.sample``, each element along the diagonal of the covariance matrix would be given an initial value of $1$.
We could then modify each data vector's covariance matrix by hand after construction of the sample (see description below).

The measurement uncertainty on any one quantity can be obtained by taking the square root of the relevant covariance matrix entry along the diagonal.
For example:

In [12]:
print(np.sqrt(data[0].cov["x"]))
print(np.sqrt(data[0].cov["x", "x"]))

0.1
0.1


Note that we received the same result with ``data[0].cov["x"]`` and ``data[0].cov["x", "x"]``.
The covariance matrix is smart enough to know that the user is looking for an element along the diagonal when it is given only one string label or integer index.

If any of the quantities covary, that information can be entered now, after the sample has been constructed.
Both strings and integers are supported; the appropriate integer corresponds to a component-wise match to the string labels returned by the ``.keys()`` function shown above (i.e. ``data[0].cov[0, 1]`` is the same as ``data[0].cov["x", "y"]``).

For example:

In [13]:
data[0].cov["x", "y"] = 0.2
print(data[0].cov)

covariance matrix(
    [ 1.0000e-02,  2.0000e-01]
    [ 2.0000e-01,  1.0000e-02]
    Quantities: [x, y] (in the order of indexing)
)


Because covariance matrices are symmetric by definition, TrackStar has taken the liberty of modifying not only ``data[0].cov["x", "y"]`` as requested, but ``data[0].cov["y", "x"]`` as well.
Any modification to a covariance matrix changes the components both above and below the diagonal automatically.

We'll simply change the value back to zero, since our mock data do not include any covariance in the input data.

In [14]:
data[0].cov["y", "x"] = 0
print(data[0].cov)

covariance matrix(
    [ 1.0000e-02,  0.0000e+00]
    [ 0.0000e+00,  1.0000e-02]
    Quantities: [x, y] (in the order of indexing)
)


### Extra Information for your ``Sample``

It is often useful for data to have additional information available that does not necessarily correspond to the measurements themselves.
One such example would be ID numbers for each data vector.
In the context of astrophysics, these may be, e.g., unique identifiers for stars from a given survey or catalog.

TrackStar allows your ``sample`` to store additional data through its ``extra`` attribute, which is a subclass of ``dict``.
To demonstrate this, we'll give each of our data vectors a name:

In [15]:
for i in range(data.size):
    data[i].extra["id"] = "name" + str(i + 1)

The above for-loop is functionally equivalent to going through the ``data.extra`` attribute to modify each entry in the sample in one fell swoop:

In [16]:
data.extra["id"] = ["name%d" % (i + 1) for i in range(data.size)]
print(data)

sample(
    N = 100
    x --------------> [ 4.2162e-01,  6.8792e-01,  1.6719e+00, ...,  7.2159e-01,  1.0742e+00, -5.5068e-01]
    y --------------> [ 5.3783e-01,  9.3261e-01,  1.9732e+00, ...,  6.7938e-01,  1.0338e+00, -6.4173e-01]
    z --------------> [ nan       ,  8.9021e-01,  nan       , ...,  1.0793e+00,  nan       ,  4.9216e-01]

    extra
    -----
    id -------------> [      name1,       name2,       name3, ...,      name98,      name99,     name100]
)


And the information can be accessed by indexing in various intuitive ways:

In [17]:
print(data[0].extra["id"])
print(data.extra["id", 0])
print(data.extra[0]["id"])

name1
name1
name1


## An Important Note on NaNs in TrackStar

While TrackStar returns ``nan`` values when it does not have a measurement for a particular quantity, they are not stored in the backend. As a consequence, **they do not actually correspond to actual blocks of memory stored on your system**.
The ``nan`` values that it returns therefore do not correspond to an existing memory address; instead, TrackStar simply tracks which measurements are available for which quantities, so it knows when it should return a ``nan``.

Based on this behavior, TrackStar does not allow users to change ``nan`` values to real numbers or vice versa. Doing so would change the dimensionality of the stored data and result in memory errors.
If new measurements are to be added to a data vector that already exists as a ``datum`` variable, one must simply make a new ``datum``.

### Beware when Indexing Non-Uniform Samples

When working with data in which some vectors do not have measurements for every quantity, as is the case in our [mock sample](#a-mock-data-sample) in the $z$-direction, a ``KeyError`` may arise if one is not careful.
The notion that *no memory is allocated for the missing quantities* is central to this behavior.
To demonstrate this behavior, consider the block of code below, where we ask TrackStar for the measurement of $z$ associated with our $0$'th datum, which has no such measurement:

In [18]:
print(data["z", 0])
print(data[0, "z"])
print(data["z"][0])
print(data[0]["z"])

nan
nan
nan


KeyError: 'Unrecognized datum label: z'

The first three times we indexed our sample, we got the expected ``nan`` value back.
However, a ``KeyError`` was raised the fourth time.
The difference between the fourth line and the preceding lines is that we *first* asked TrackStar for the $0$th datum, and *then* asked that datum for its measurement of $z$, for which there is none.
Unaware that it is embedded in a sample that contains measurements of these quantities, it raises an error.
In the third line, we first asked the sample as a whole for every $z$ measurement, so TrackStar knew to include a ``nan`` for the $0$'th datum.
In the second and third lines, we gave the sample both ``"z"`` and ``0`` simultaneously, so it immediately knew that we were asking for a measurement that does not exist.

As discussed above, it is both faster and more memory efficient to index TrackStar data with all values simultaneously.
Consequently, ``sample[0, "z"]`` and ``sample["z", 0]`` are recommended over ``sample["z"][0]`` anyway.